# The `QuantumState` class

Express an N-qubit quantum state as an array of N columns, where rows are binary bit values, complemented by a vector of coefficients. Note the similarity with the symplectic representation - indeed, the QuantumState class will infact set the above array as the X block in a PauliwordOp, with the Z block its complement.

What we are doing here is writing $|0\rangle = Z|0\rangle$ and $|1\rangle = X|0\rangle$, which ensures correct phases when multiplying the state by Pauli operators, since

$$X|0\rangle = XZ|0\rangle = -iY|0\rangle = |1\rangle,\; X|1\rangle = XX|0\rangle = |0\rangle$$
$$Y|0\rangle = YZ|0\rangle = iX|0\rangle = i|1\rangle,\; Y|1\rangle = YX|0\rangle = -iZ|0\rangle = -i|0\rangle$$
$$Z|0\rangle = ZZ|0\rangle = |0\rangle,\; Z|1\rangle = ZX|0\rangle = iY|0\rangle = -|1\rangle$$

Finally, we have $$| \vec{b} \rangle = \bigotimes_{i=1}^N \big(b_i X + (1-b_i) Z\big) | \vec{0} \rangle$$ and we may drop the zero vector and use the functionality of PauliwordOp to manipulate quantum states. In this represenation, a quantum state is stored as an operator consisting of Paulis $X, Z$, which is implicitly applied to the zero (or vacuum) state. 

In [1]:
import numpy as np
from symmer.symplectic import PauliwordOp, QuantumState

Firstly, note the correct phases under multiplication by a Pauli Y:

In [2]:
zero = QuantumState([[0]])
one = QuantumState([[1]])
Y = PauliwordOp.from_list(['Y'], [1])

print(f'{zero} -> {Y * zero}')
print(f'{one} -> {Y * one}')

 1.000+0.000j |0> ->  0.000+1.000j |1>
 1.000+0.000j |1> ->  0.000-1.000j |0>


Now, let's see what happens when we apply the Hadamard gate to the zero state:

In [3]:
psi = QuantumState([[0,0]])
had = PauliwordOp.from_list(['XZ','ZX','XX', 'ZZ'], np.ones(4)/2) # 2-qubit Hadamard gate decomposed over Paulis
eq_superposition = had * psi
print(f'Zero state: {psi}\n')
print(f'After application of the Hadamard gate:\n\n{eq_superposition}')

Zero state:  1.000+0.000j |00>

After application of the Hadamard gate:

 0.500+0.000j |00> +
 0.500+0.000j |01> +
 0.500+0.000j |10> +
 0.500-0.000j |11>


Observe that the QuantumState is represented internally by its `state_op`, a PauliwordOp object that governs its behaviour under multiplication

In [4]:
print(eq_superposition.state_op)

 0.500+0.000j ZZ +
 0.500+0.000j ZX +
 0.500+0.000j XZ +
 0.500-0.000j XX


Try evaluating expectation values for randomly generated state and Hermitian operators:

In [5]:
def random_state(num_qubits, num_terms):
    """ Generates a random normalized QuantumState
    """
    # random binary array with N columns, M rows
    random_state = np.random.randint(0,2,(num_terms,num_qubits))
    # random vector of coefficients
    coeff_vec = (
        np.random.rand(num_terms) + 
        np.random.rand(num_terms)*1j
    )
    return QuantumState(random_state, coeff_vec).normalize

def random_herm_op(num_qubits, num_terms):
    """ Generates a random PaulwordOp
    """
    # random binary array with 2N columns, M rows
    random_symp_mat = np.random.randint(0,2,(num_terms,num_qubits*2))
    # random vector of coefficients
    coeff_vec = np.random.rand(num_terms)
    coeff_vec/=coeff_vec[::-1]
    coeff_vec*=(2*np.random.randint(0,2,num_terms)-1)
    return PauliwordOp(random_symp_mat, coeff_vec)

In [6]:
N = 10 # number of qubits
M = 10 # number of terms

psi_1 = random_state(N, M)
psi_2 = random_state(N, M)

print(psi_1)
print()
print(psi_2)

 0.257+0.050j |1010010011> +
 0.152+0.293j |1110000001> +
 0.339+0.364j |0100001101> +
 0.201+0.013j |1110000111> +
 0.322+0.239j |0011100110> +
 0.047+0.252j |0100010100> +
 0.047+0.011j |1101001000> +
 0.107+0.314j |1000010010> +
 0.296+0.094j |0111001000> +
 0.253+0.191j |0010011101>

 0.134+0.117j |1101101100> +
 0.263+0.270j |1111011100> +
 0.250+0.182j |1000011011> +
 0.173+0.064j |1000100100> +
 0.025+0.303j |0100001101> +
 0.343+0.332j |0110110011> +
 0.021+0.298j |1111111101> +
 0.229+0.004j |0111011101> +
 0.312+0.041j |0000101000> +
 0.220+0.296j |1000111001>


In [9]:
psi = (psi_1 + psi_2).normalize
print(psi)
print()
print('Norm:', psi.dagger * psi)

 0.209+0.027j |0000101000> +
 0.169+0.128j |0010011101> +
 0.215+0.160j |0011100110> +
 0.243+0.446j |0100001101> +
 0.032+0.168j |0100010100> +
 0.229+0.222j |0110110011> +
 0.198+0.063j |0111001000> +
 0.153+0.003j |0111011101> +
 0.072+0.210j |1000010010> +
 0.167+0.121j |1000011011> +
 0.115+0.043j |1000100100> +
 0.147+0.198j |1000111001> +
 0.171+0.034j |1010010011> +
 0.032+0.007j |1101001000> +
 0.090+0.078j |1101101100> +
 0.102+0.196j |1110000001> +
 0.134+0.009j |1110000111> +
 0.176+0.180j |1111011100> +
 0.014+0.200j |1111111101>

Norm: (0.9999999999999998+0j)


In [11]:
op = random_herm_op(N, 10)
print(op)
print()
print('Expectation value <psi|op|psi> =', psi.dagger * op * psi)

 1.876+0.000j YYIZYIXIYI +
 1.386+0.000j YZZYZIIYYY +
-0.155+0.000j ZIYZYIZXZY +
-4.208+0.000j XYIXZIZXZZ +
-0.297+0.000j IIIIYXZYZI +
 3.366+0.000j YYZZXIIXYY +
 0.238+0.000j XYIIXYIIZI +
 6.436+0.000j YIIYIYYIIX +
-0.722+0.000j YYIYIXIZIX +
-0.533+0.000j XZZYIYYZXZ

Expectation value <psi|op|psi> = (-0.12298367631617613+6.938893903907228e-18j)


It is also possible to convert from a $2^N$ dimensional statevector over $N$ qubits to a `QuantumState` object, via the `array_to_QuantumState` function:

In [13]:
statevector = np.zeros(2**10).reshape([-1,1])
statevector[0]   = np.sqrt(1/6)
statevector[250] = np.sqrt(2/6)
statevector[500] = np.sqrt(3/6)
psi = QuantumState.from_array(statevector)
print(psi)
print(psi.dagger * psi)

 0.408+0.000j |0000000000> +
 0.577+0.000j |0011111010> +
 0.707+0.000j |0111110100>
(1+0j)
